# Create Public School Machine Learning Datasets
** This program creates all the _ML datasets in the NCEA repository.** 
* This notebook reads each School Dataset file located at \EducationDataNC\ *schoolYear* \School Datasets\ as input data.
* Different school years are processed by changing the *schoolYear* parameter.
* Different input / output files are processed / created by changing the *inputFileName* paramter in the cell below.  
* While a single program is used to create all the _ML datasets, one program copy per dataset is maintained in the repositiory so the dataset specific tranformation reports may be reviewed. 

**Datasets ending in ML are preprocessed for Machine Learning and go through the following transformations: **
1. Missing student body racial compositions are imputed using district averages.
2. Columns that have the same value in every single row are deleted.
3. Columns that have a unique value in every single row (all values are different) are deleted.
4. Empty columns (all values are NA or NULL) are deleted.
5. Numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
6. Remaining numeric, non-race columns with missing values are imputed / populated with 0.  In many cases, schools are not reporting values when they are zero. However, mean imputation or some other more sophisticated strategy might be considered here.
7. Categorical / text based columns with > *uniqueThreshold* unique values are deleted.
8. All remaining categorical / text based columns are one-hot encoded.  In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
9. Duplicated or highly similar columns with > 95% correlation are delelted.    

In [1]:
#import required Libraries
import pandas as pd
import numpy as np
import os
import string

#**********************************************************************************
# Set the following variables before running this code!!!
#**********************************************************************************
#All raw data files are processed for the year below
schoolYear = 2015

#Location where copies of the raw data files will be read in from csv files.
dataDir = '../../../' + str(schoolYear) + '/School Datasets/'

#Name of the file to be processed
inputFileName = 'PublicSchools' + str(schoolYear)
#inputFileName = 'PublicHighSchools' + str(schoolYear)
#inputFileName = 'PublicMiddleSchools' + str(schoolYear)
#inputFileName = 'PublicElementarySchools' + str(schoolYear)

#Input file being transformed for machine learning 
inputFile = dataDir + inputFileName + '.csv'

#Location where the new school datasets will be created.
outputDir = '../../../' + str(schoolYear) + '/Machine Learning Datasets/'

#Missing Data Threshold (Per Column)
missingThreshold = 0.60

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 25

#Read in the School Data File
schoolData = pd.read_csv(inputFile, low_memory=False, dtype={'unit_code': object})
print('*********Start: Beginning Column and Row Counts********************************************')
schoolData.info(verbose=False)

#Select only public schools as charter schools are missing data for many columns.
schoolData = schoolData[(schoolData['type_cd'] == 'P') & (schoolData['student_num'] > 0)]

print('\r\n*********After: Selecting Only Public School Campuses**********************************')
schoolData.info(verbose=False)

#Save primary key
unit_code = schoolData['unit_code']
#Convert zip code to string
schoolData['szip_ad'] = schoolData['szip_ad'].astype('object')
#Rename Lea_Name to District Name
schoolData.rename(columns={'Lea_Name': 'District Name'}, inplace=True)

*********Start: Beginning Column and Row Counts********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2585 entries, 0 to 2584
Columns: 2143 entries, vphone_ad to Attrtion_Reason_OtherReasonsPerct
dtypes: float64(2090), int64(3), object(50)
memory usage: 42.3+ MB

*********After: Selecting Only Public School Campuses**********************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 2143 entries, vphone_ad to Attrtion_Reason_OtherReasonsPerct
dtypes: float64(2090), int64(3), object(50)
memory usage: 39.8+ MB


# Prepare Consolidated Dataset for Machine Learning
**Below we perform operations on the entire dataset to remove columns and update row values that could cause problems during machine learning.**

## Student Body Racial Composition Features 
**Impute / update missing Student Body Racial Composition Fields using mean imputation.**
* When there are no racial composition percentages for a particular school campus / unit_code, fill in the missing values 

In [2]:
#schoolData.info(verbose=True)

In [3]:
#Get Student Body Racial Composition Fields
raceCompositionFields = schoolData.filter(regex='Indian|Asian|Hispanic|Black|White|PacificIsland|TwoOrMore|Minority')\
                                  .filter(regex='Pct').columns
    
rowsBefore = schoolData[raceCompositionFields].isnull().T.any().T.sum()

#Update missing race values with the district average when avaiable (No district averages for charter schools) 
schoolData[raceCompositionFields] = schoolData.groupby('District Name')[raceCompositionFields]\
                                              .transform(lambda x: x.fillna(x.mean()))

    #Review dataset contents after Racial Composition Imputation
print('*********After: Updating Missing Racial Compostion Values****************************')   
rowsAfter = schoolData[raceCompositionFields].isnull().T.any().T.sum()
rowsUpdated = rowsBefore - rowsAfter
print ('Rows Updated / Imputed: ', rowsUpdated )
print('\r\nTotal Rows Missing Racial Compositions By District Name') 
schoolData['District Name'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts()

*********After: Updating Missing Racial Compostion Values****************************
Rows Updated / Imputed:  0

Total Rows Missing Racial Compositions By District Name


Series([], Name: District Name, dtype: int64)

## Remove Columns with Problematic Data
**Here we remove entire columns that could cause problems during machine learning.  The following operations are performed:**
* Remove any columns that have the same value in every single row.
* Remove any columns that have a unique value in every single row (all values are different).
* Remove empty columns (all values are NA or NULL).

In [4]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 1954 entries, vphone_ad to Attrtion_Reason_OtherReasonsPerct
dtypes: float64(1908), object(46)
memory usage: 36.3+ MB

Columns Deleted:  189


In [5]:
#Remove any fields that have unique values in every row
schoolDataRecordCt = schoolData.shape[0]
UniqueValueCounts = schoolData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
schoolData.info(verbose=False)
print( '\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 1952 entries, vphone_ad to Attrtion_Reason_OtherReasonsPerct
dtypes: float64(1908), object(44)
memory usage: 36.2+ MB

Columns Deleted:  2


In [6]:
#Remove any empty fields (null values in every row)
schoolDataRecordCt = schoolData.shape[0]
NullValueCounts = schoolData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 1952 entries, vphone_ad to Attrtion_Reason_OtherReasonsPerct
dtypes: float64(1908), object(44)
memory usage: 36.2+ MB

Columns Deleted:  0


## Handle Other Missing Values Types
* Here we eliminate any numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
* All remaining non-race, numeric column missing values are populated with 0.
* In many cases, it seems that schools are not simply not reporting values when they are zero. However, mean imputation or some other strategy might be considered.

In [7]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print( "Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  44
Continuous Columns:  1908
Columns Accounted for:  1952


In [8]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 1454 entries, vphone_ad to Attrtion_Reason_OtherReasonsPerct
dtypes: float64(1410), object(44)
memory usage: 27.0+ MB

Columns Deleted:  498


## One-Hot Encoding of Categorical Variables
**All categorical / string variables are converted to numberic variables via one hot encoding.  Each unique row value will become a new binary / numeric column in the dataset.**
* All remaining categorical columns are one-hot encoded.  
* In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
* Prior to one-hot encoding, columns with > *uniqueThreshold* unique values are deleted.  

In [9]:
#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 1434 entries, closed_ind to Attrtion_Reason_OtherReasons
dtypes: float64(1410), object(24)
memory usage: 26.6+ MB

Columns Deleted:  20


In [10]:
sD_nominal.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Data columns (total 44 columns):
vphone_ad                                   2432 non-null object
street_ad                                   2432 non-null object
scity_ad                                    2432 non-null object
szip_ad                                     2432 non-null object
category_cd                                 2432 non-null object
url_ad                                      2358 non-null object
grade_range_cd                              2432 non-null object
calendar_type_txt                           2429 non-null object
sna_pgm_type_cd                             2429 non-null object
cover_letter_ad                             1784 non-null object
school_type_txt                             2429 non-null object
calendar_only_txt                           2426 non-null object
title1_type_cd                              1322 non-null object
clp_ind                                     117 

In [11]:
sD_nominal.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Data columns (total 44 columns):
vphone_ad                                   2432 non-null object
street_ad                                   2432 non-null object
scity_ad                                    2432 non-null object
szip_ad                                     2432 non-null object
category_cd                                 2432 non-null object
url_ad                                      2358 non-null object
grade_range_cd                              2432 non-null object
calendar_type_txt                           2429 non-null object
sna_pgm_type_cd                             2429 non-null object
cover_letter_ad                             1784 non-null object
school_type_txt                             2429 non-null object
calendar_only_txt                           2426 non-null object
title1_type_cd                              1322 non-null object
clp_ind                                     117 

In [12]:
#Isolate remaining categorical variables
begColumnCt = len(schoolData.columns)
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]

#one hot encode categorical variables
schoolData = pd.get_dummies(data=schoolData, columns=sD_nominal.columns, drop_first=True)

#Determine change in column count
endColumnCt = len(schoolData.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print ('Columns To One-Hot Encode: ', len(sD_nominal.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
schoolData.info(verbose=False)
print( '\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

Columns To One-Hot Encode:  24

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 1490 entries, closed_ind to lea_avg_age_media_collection_2010
dtypes: float64(1410), uint8(80)
memory usage: 26.4 MB

New Columns Created Via One-Hot Encoding:  56


## Impute any Remaining Missing Values as Zero

In [13]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values




*********The Remaining Missing Values Below will be set to Zero!*************************


,Variable Name,Number Missing Values
4,00_Size,1179
5,01_Size,1195
6,02_Size,1195
7,03_Size,1194
8,04_Size,1168
9,05_Size,1192
10,Math I_Size,1397
20,SPG Score,129
21,Reading SPG Score,604
22,Math SPG Score,604


In [14]:
#Replace all remaining NaN with 0
schoolData = schoolData.fillna(0)

#Check for Missing values after final imputation 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

,Variable Name,Number Missing Values


## Identify and Remove Highly Correlated Features
**Find and remove any columns / features that are > 95% correlated**
* https://stackoverflow.com/questions/39409866/correlation-heatmap
* https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
* https://codeyarns.com/2015/04/20/how-to-change-font-size-in-seaborn/

In [15]:
# calculate the correlation matrix
corr_matrix  = schoolData.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [16]:
#Get all of the correlation values > 95%
x = np.where(upper > 0.95)

#Display all field combinations with > 95% correlation
cf = pd.DataFrame()
cf['Field1'] = upper.columns[x[1]]
cf['Field2'] = upper.index[x[0]]

#Get the correlation values for every field combination. (There must be a more pythonic way to do this!)
corr = [0] * len(cf)
for i in range(0, len(cf)):
    corr[i] =  upper[cf['Field1'][i]][cf['Field2'][i]] 
    
cf['Correlation'] = corr

print( 'There are ', str(len(cf['Field1'])), ' field correlations > 95%.')
cf

There are  215192  field correlations > 95%.


,Field1,Field2,Correlation
0,class_teach_num,student_num,0.966997
1,lea_class_teach_num,lea_avg_student_num,0.973656
2,st_class_teach_num,st_avg_student_num,0.995897
3,st_nbpts_num,st_avg_student_num,0.957187
4,st_advance_dgr_pct,st_avg_student_num,0.980537
5,Overall Achievement Score,SPG Score,0.986319
6,Math SPG Score,Reading SPG Score,0.974374
7,Read Score,Reading SPG Score,0.992346
8,Math Score,Reading SPG Score,0.957245
9,EOGMathGr3-8_GLP_All,Reading SPG Score,0.950945


In [17]:
print( 'Dropping the following ', str(len(to_drop)), ' highly correlated fields.')
to_drop

Dropping the following  1086  highly correlated fields.


['Math SPG Score',
 'Overall Achievement Score',
 'Read Score',
 'Math Score',
 'EOCMathI_CACR_All',
 'EOCMathI_GLP_All',
 'EOCSubjects_CACR_All',
 'EOCSubjects_GLP_All',
 'EOG/EOCSubjects_GLP_All',
 'EOGGr3_GLP_All',
 'EOGGr4_GLP_All',
 'EOGGr5_GLP_All',
 'EOGMathGr3_CACR_All',
 'EOGMathGr3_GLP_All',
 'EOGMathGr3-8_CACR_All',
 'EOGMathGr3-8_GLP_All',
 'EOGMathGr4_CACR_All',
 'EOGMathGr4_GLP_All',
 'EOGMathGr5_CACR_All',
 'EOGMathGr5_GLP_All',
 'EOGReadingGr3_CACR_All',
 'EOGReadingGr3_GLP_All',
 'EOGReadingGr3-8_CACR_All',
 'EOGReadingGr3-8_GLP_All',
 'EOGReadingGr4_CACR_All',
 'EOGReadingGr4_GLP_All',
 'EOGReadingGr5_CACR_All',
 'EOGReadingGr5_GLP_All',
 'EOGScienceGr5_CACR_All',
 'EOGScienceGr5_GLP_All',
 'EOGScienceGr5&8_CACR_All',
 'EOGScienceGr5&8_GLP_All',
 'EOGSubjects_CACR_All',
 'EOGSubjects_GLP_All',
 'EOCMathI_CACR_Female',
 'EOCMathI_GLP_Female',
 'EOCSubjects_CACR_Female',
 'EOCSubjects_GLP_Female',
 'EOG/EOCSubjects_CACR_Female',
 'EOG/EOCSubjects_GLP_Female',
 'EOGGr3_C

In [18]:
#Check columns before drop 
print('\r\n*********Before: Dropping Highly Correlated Fields*************************************')
schoolData.info(verbose=False)

# Drop the highly correlated features from our training data 
schoolData = schoolData.drop(to_drop, axis=1)

#Check columns after drop 
print('\r\n*********After: Dropping Highly Correlated Fields**************************************')
schoolData.info(verbose=False)


*********Before: Dropping Highly Correlated Fields*************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 1490 entries, closed_ind to lea_avg_age_media_collection_2010
dtypes: float64(1410), uint8(80)
memory usage: 26.4 MB

*********After: Dropping Highly Correlated Fields**************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 404 entries, closed_ind to lea_avg_age_media_collection_2010
dtypes: float64(344), uint8(60)
memory usage: 6.5 MB


In [19]:
#Restore the unit_code before saving
schoolData['unit_code'] = unit_code
#Save the final dataset to a .csv file
schoolData.to_csv(outputDir + inputFileName + '_ML.csv', sep=',', index=False)

In [20]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
schoolData.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Data columns (total 405 columns):
closed_ind                                                                         float64
student_num                                                                        float64
lea_avg_student_num                                                                float64
st_avg_student_num                                                                 float64
00_Size                                                                            float64
01_Size                                                                            float64
02_Size                                                                            float64
03_Size                                                                            float64
04_Size                                                                    

In [21]:
import sklearn
import pandas as pd

print('Sklearn Version: ' + sklearn.__version__)
print('Pandas Version: ' + pd.__version__)

Sklearn Version: 0.20.1
Pandas Version: 0.24.2


In [22]:
print ('Output File Location:\r\n\r\n' + outputDir + inputFileName + '_ML.csv')

Output File Location:

../../../2015/Machine Learning Datasets/PublicSchools2015_ML.csv
